## Paillier 同态密码系统简介及其 Python 实现
* 同态，通俗理解就是运算方可以在不知道运算数明文的情况下对密文进行某种计算，使得计算的结果仍为合法的密文，且解密后恰好等于原两明文的某种计算的结果。
* 假设加密算法为 $E(x)$，其中 $x$ 为明文，则同态密码系统可以表示为符合以下性质：
  $$E(x) \oplus E(y) = E(x + y) $$
  其中，$x, y$ 为明文， $\oplus$ 是密文上的某种运算，$+$ 是明文上的某种运算，通常为加法或乘法。
* 全同态密码系统和半同态密码系统：当一个同态密码系统同时支持对密文进行运算实现明文上的加法和乘法时，该密码系统称为全同态密码系统；而仅支持加法或乘法中的一种时，称为半同态密码系统。
* Paillier 同态密码系统
  * Paillier同态密码系统是半同态密码系统，支持同态加法运算。

In [ ]:
import random
import math

# quick power: calculate (base^exponent)%modulus
def powerMod(base, exponent, modulus):
  answer = 1
  while exponent > 0:
    if exponent % 2 == 1: answer = (answer * base) % modulus
    base = (base**2) % modulus
    exponent //= 2
  return answer

# exgcd: return (x, y, gcd(a, b)) where ax + by = gcd(a,b)
def exgcd(a, b):
  if b == 0: return 1, 0, a
  else:
    x0, y0, g = exgcd(b, a%b)
    return y0, x0 - (a//b)*y0, g

# inv: return x, where ax = 1 (mod m)
def inv(a, m) -> int:
    x, y, g = exgcd(a, m)
    return (x%m+m)%m

* 首先我们需要找出一些较大的质数。
  * 在这里为了简便我们直接使用埃拉托色尼筛法。在实际应用中为了选出随机大素数我们可以使用 Miller Rabin 素性检测的方法。
* 找出素数后我们输出找到的最大的几个素数以便检查正确性。

In [ ]:
def sieve(upperbound = 0x4000):
  primes = []
  flags = [True] * upperbound
  for each in range(2, upperbound):
    if not flags[each]: continue
    for multiplier in range(2, upperbound // each):
      flags[multiplier * each] = False
    primes.append(each)
  return primes

primes = sieve(0x4000)
print(primes[-10:])

* 密钥生成
  1. 选取两个随机大素数 $p, q$，计算 $n = pq, \lambda = \text{lcm}(p-1, q-1)$；
  2. 选取 $g < n^2$ 且与 $n^2$ 互质（即 $g$ 不为 $p$ 或 $q$ 的倍数），使模 $n$ 意义下的逆元 $\mu = (L(g^\lambda \text{ mod } n^2))^{-1}$ 存在，即
      $$\mu \cdot L(g^\lambda \text{ mod } n^2) \equiv 1 \pmod {n}$$
      其中 $L(x) = (x-1)/n$；
  3. 公钥为 $(n, g)$，私钥为 $(\lambda, \mu)$。从实现的角度而言，因为 $g$ 是公钥，所以不必选取 $g$ 为随机数，例如可以直接选取 $g = n+1$。

In [ ]:
# produce (n, g, lambda, mu), where (n, g) is the public key, (lambda, mu) is the private key
def generateKeys():
  primeCount = len(primes)
  p = primes[random.randint(primeCount // 2, primeCount)]
  while True:
    q = primes[random.randint(primeCount // 2, primeCount)]
    if p != q: break
  n = p*q
  Lambda = (p-1)*(q-1) // math.gcd(p-1, q-1)
  g = n + 1
  mu = inv((powerMod(g, Lambda, n*n)-1)//n, n)
  return n, g, Lambda, mu

* 加密算法
  * 对于明文 $m < n$ 随机选取 $0 < r < n$ 使得 $r$ 与 $n$ 互质，则密文为 $c = g^m r^n (\text{mod } n^2)$
  * 实际上，当 $n$ 足够大时，可以直接随机选取 $0 < r < n$，因为二者不互质的概率极小。

In [ ]:
def encrypt(m, n, g):
  while True:
    r = random.randint(1, n-1)
    if math.gcd(r, n) == 1: break
  c = powerMod(g, m, n*n) * powerMod(r, n, n*n) % (n*n)
  return c

* 解密算法
  * 对于密文 $c$，明文为 $m = \mu \cdot L(c^\lambda \text{ mod } n^2) \text{ mod } n$

In [ ]:
def decrypt(c, Lambda, mu, n):
  k = powerMod(c, Lambda, n*n)
  assert((k-1)%n == 0) # when (k-1)%n != 0, c is not a valid ciphertext.
  return (k-1)//n * mu % n  

* 同态加法运算
  * 对于密文 $c_1, c_1$, 计算 $c_3 = c_1 \cdot c_2 \text{ mod } n^2$，则 $c_3$ 是合法的密文且
    $$D(c_3) = D(c_1) + D(c_2)$$
    其中 $D(c)$ 为解密算法。
    

In [ ]:
def evalAdd(c1, c2, n):
  return c1 * c2 % (n*n)

* 我们可以测试以上实现是否确实满足同态性质。

In [ ]:
# generate keys
n, g, Lambda, mu = generateKeys()
print(f"Public key:       n = {n:10d},  g = {g:10d}")
print(f"Private key: lambda = {Lambda:10d}, mu = {mu:10d}")
print("")
# plaintext
m1 = random.randint(0, n-1)
m2 = random.randint(0, n-1)
# ciphertext
c1 = encrypt(m1, n, g)
c2 = encrypt(m2, n, g)
print(f"c1 = Encrypt({m1}) = {c1:18d} = 0x{c1:015x}")
print(f"c2 = Encrypt({m2}) = {c2:18d} = 0x{c2:015x}")
print("")
# evaluate addition
c3 = evalAdd(c1, c2, n)
print(f"c3 = c1 * c2 = {c3:18d} = 0x{c3:015x}")
print("")
# decrypt
d = decrypt(c3, Lambda, mu, n)
print(f"Decrypt(c3) = {d} = {m1} + {m2} (mod {n})")


* Paillier 密码系统中加密和解密的正确性

    由 $\lambda = \text{lcm}(p-1, q-1)$，可记 $\lambda = k_1(p-1) = k_2(q-1)$。

    因 $g$ 不是 $p$ 的倍数，由费马小定理可知 $g^{\lambda} = g^{k_1 (p-1)} \equiv 1 \pmod{p}$；同理 $g^{\lambda} \equiv 1 \pmod{q}$；从而 $g^\lambda \equiv 1 \pmod{n}$，即 $g^\lambda \text{ mod } n^2 \equiv 1 \pmod{n}$。记 $g^\lambda \text{ mod } n^2 = kn + 1$，即 $L(g^\lambda \text{ mod } n^2) = k$。

    由二项式定理，$(1 + kn)^m \equiv knm + 1 \pmod{n^2}$，从而 $g^{m\lambda} \equiv (kn+1)^m \equiv knm + 1 \pmod{n^2}$。

    同样，因为 $\gcd(r, n) = 1$，则 $r^\lambda \equiv 1$，记为 $r^\lambda = k_r n + 1$，则 $r^{\lambda n} \equiv k_r n^2 + 1 \equiv 1 \pmod {n^2}$。

    于是 $L(g^{m\lambda}r^{n\lambda} \text{ mod } n^2) = L(knm + 1) = km$，从而 $\mu L(g^{m\lambda}r^{n\lambda} \text{ mod } n^2) \equiv km / k \equiv m \pmod{n}$。

* 同态加法的正确性
  
    设两密文 $c_1 = g^{m_1}r_1^n \text{ mod } n^2, c_2 = g^{m_2}r_2^n \text{ mod } n^2$，则 $c_3 \equiv c_1c_2  \equiv g^{m_1+m_2} r_1^n r_2^n \pmod{n^2}$。

    由以上分析可知 $g^{(m_1+m_2)\lambda} \equiv kn(m_1+m_2) + 1$，而 $r_1^{\lambda n} \equiv r_2^{\lambda n} \equiv 1 \pmod{n^2}$，易得 $\mu L(c_3^\lambda  \text{ mod } n^2) \equiv k(m_1+m_2) / k \equiv m_1 + m_2 \pmod{n}$。即加法同态成立。
  